<a href="https://colab.research.google.com/github/imVariable/imVariable/blob/main/Youtube_NLP_Text_Representation_Kaggle_Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [61]:
df = pd.read_csv(r"/content/drive/MyDrive/Dataset/train.csv", error_bad_lines=False)
df = df[['comment_text','toxic']].drop_duplicates().reset_index(drop=True)
df = df.iloc[:2000,:]
print(df.head())

<ipython-input-61-0709090d2ea3>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(r"/content/drive/MyDrive/Dataset/train.csv", error_bad_lines=False)


                                        comment_text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0


In [62]:
##### Removing un-necessary or stop words using nlp spacy models.
nlp = spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'which', 'whither', 'thereupon', 'anyway', 'can', 'since', "'m", 'anything', 'give', 'i', 'hers', 'last', 'after', 'did', '‘s', 'might', 'thereafter', '‘re', 'yourself', 'before', 'two', 'somewhere', 'their', 'whose', 're', 'back', '‘ll', 'then', 'all', 'whereafter', 'hundred', 'had', 'see', 'now', 'under', 'make', 'nine', 'anyone', 'along', 'via', 'ten', 'you', 'those', 'again', 'bottom', 'rather', 'have', 'should', 'will', 'someone', 'for', 'empty', 'amongst', 'who', 'become', 'other', 'often', 'do', 'front', 'no', 'cannot', "'re", 'between', 'eight', 'whether', 'hence', 'well', 'thereby', 'him', 'each', 'really', 'using', 'among', 'regarding', 'hereby', 'every', 'everywhere', 'unless', 'whence', 'he', 'us', 'one', 'thence', 'thru', 'please', 'beside', 'how', 'everyone', 'formerly', 'themselves', 'full', 'herein', 'not', 'they', 'here', 'ca', 'four', 'across', 'whereby', 'them', 'part', 'our', 'besides', 'from', 'call', '‘ve', 'into', 'n‘t', 'more', 'forty', 'the', 'except', 'your',

In [63]:
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [64]:
## Creating tokenizer function
def spacy_tokenizer(sentence):
  ## Creating our token object, which is used to create documents with linguistic annotations
  doc = nlp(sentence)
  print(doc)

  #### Lemmatizing each token and converting each token into lowercase
  mytokens = [ word.lemma_.lower().strip() for word in doc ]
  # print(mytokens)

  ## Removing stop words
  mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations ]

  ## return preprocessed list of tokens
  return mytokens



In [65]:
sentence = "Soni has been eating bananas."
spacy_tokenizer(sentence)

Soni has been eating bananas.


['soni', 'eat', 'banana']

In [66]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)

In [67]:
count_vector.fit_transform(["I've slept early in the morning",'Ram is going to school.']).toarray()

i've slept early in the morning
ram is going to school.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[1, 1, 0, 0, 1],
       [0, 0, 1, 1, 0]])

In [68]:
## Features/words considered for tokenizer
count_vector.get_feature_names_out()

array(['early', 'morning', 'ram', 'school', 'sleep'], dtype=object)

In [69]:
count_vector.vocabulary_

{'sleep': 4, 'early': 0, 'morning': 1, 'ram': 2, 'school': 3}

In [70]:
from sklearn.model_selection import train_test_split

X = df['comment_text']
ylabels = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size = 0.2)

In [71]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [72]:
X_train_vectors = count_vector.fit_transform(X_train)
X_test_vectors = count_vector.transform(X_test)

Streaming output truncated to the last 5000 lines.
the turks (or turkics) are peoples residing in northern, central and western asia, mongolia, southern siberia and northwestern china and parts of eastern europe. they speak languages belonging to the turkic language family.turkic people, encyclopædia britannica, online academic edition, 2010 they share, to varying degrees, certain cultural traits and historical backgrounds. the term turkic represents a broad ethno-linguistic group of people including existing societies such as the  azerbaijanis, chuvashes, kazakhs, tatars, kyrgyzs, turkish, turkmen, uyghur, uzbeks, bashkirs, qashqai, gagauzs, yakuts, crimean karaites, krymchaks, karakalpaks, karachays, nogais and as well as past civilizations such as the  kumans, kipchaks, avars, turgeshs, seljuks, khazars, ottomans, mamluks, timurids, and possibly the xiongnu and huns.""timur"", the columbia encyclopedia, sixth edition, 2001–05, columbia university press.encyclopaedia britannica artic

In [73]:
X_train_vectors.shape


(1600, 10780)

In [74]:
X_test_vectors.shape

(400, 10780)

In [75]:
classifier.fit(X_train_vectors, y_train)

LogisticRegression()

In [76]:
predict = classifier.predict(X_test_vectors)
print("Accuracy : ", metrics.accuracy_score(y_test, predict))
print("Precision : ", metrics.precision_score(y_test, predict))
print("Recall : ", metrics.recall_score(y_test, predict))

Accuracy :  0.9025
Precision :  0.7777777777777778
Recall :  0.2857142857142857


In [77]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [78]:
X_train_vector = tfidf_vector.fit_transform(X_train)
X_test_vector = tfidf_vector.transform(X_test)

raffaele's request for a lawyer while being interrogated on nov. 5 was refused, as was his request that he be allowed to call his father. page 138

amanda was denied food, water, bathroom breaks and naps during the all night interrogation on nov. 5-6.. page 151

wikid: actually, now looking at murder in italy, i see that there is a ton of additional information about both amanda and raffaele's interrogations that could be included. so i will work on listing all that in more detail tomorrow.
"==deletion of account==

ok blueboy, ""editing from l31 g0ng l41 has been disabled by blueboy96 for the following reason(s):

came back from 31-hour block and picked it right back up again ... and judging by history, not likely to get it""

for this:

""this is to call attention to a topic which was ""resolved"" before any of the matters were addressed.

there are moderators deleting wikipedia history items, and moderator borgqueen's edit of ""cuisine of hawaii"" was simply a provocation and nothin

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Streaming output truncated to the last 5000 lines.
the turks (or turkics) are peoples residing in northern, central and western asia, mongolia, southern siberia and northwestern china and parts of eastern europe. they speak languages belonging to the turkic language family.turkic people, encyclopædia britannica, online academic edition, 2010 they share, to varying degrees, certain cultural traits and historical backgrounds. the term turkic represents a broad ethno-linguistic group of people including existing societies such as the  azerbaijanis, chuvashes, kazakhs, tatars, kyrgyzs, turkish, turkmen, uyghur, uzbeks, bashkirs, qashqai, gagauzs, yakuts, crimean karaites, krymchaks, karakalpaks, karachays, nogais and as well as past civilizations such as the  kumans, kipchaks, avars, turgeshs, seljuks, khazars, ottomans, mamluks, timurids, and possibly the xiongnu and huns.""timur"", the columbia encyclopedia, sixth edition, 2001–05, columbia university press.encyclopaedia britannica artic

In [79]:
classifier = LogisticRegression()
classifier.fit(X_train_vectors, y_train)

LogisticRegression()

In [80]:
predict = classifier.predict(X_test_vectors)
print("Accuracy of Logistic Regression : ", metrics.accuracy_score(y_test, predict))
print("Precision of Logistic Regression : ", metrics.precision_score(y_test, predict))
print("Recall of Logistic Regression : ", metrics.recall_score(y_test, predict))


Accuracy of Logistic Regression :  0.9025
Precision of Logistic Regression :  0.7777777777777778
Recall of Logistic Regression :  0.2857142857142857
